In [ ]:


!pip install ultralytics
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 30.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
from easyocr import Reader
import torch
import cv2
import os
import csv
import json
from datetime import datetime

# ---------- CONFIG ----------
CONFIDENCE_THRESHOLD = 0.5
BOX_COLOR = (0, 255, 0)   # Green box
TEXT_COLOR = (255, 0, 0)  # Blue OCR text

# Create timestamped main folder
current_run_time = datetime.now().strftime("%Y%m%d_%H%M%S")
MAIN_SAVE_DIR = f"/content/drive/MyDrive/MinorProject/Results_csv_Json/{current_run_time}_Detect_recognise"

# Subfolders
CSV_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, "CSV")
JSON_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, "JSON")
CROPPED_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, "Cropped_numberplate")

# Ensure directories exist
os.makedirs(CSV_SAVE_DIR, exist_ok=True)
os.makedirs(JSON_SAVE_DIR, exist_ok=True)
os.makedirs(CROPPED_SAVE_DIR, exist_ok=True)

# Result storage
results_csv = []
results_json = []

# ---------- DETECTION ----------
def detect_number_plates(image, model, current_time=""):
    detections = model.predict(image)[0].boxes.data
    number_plate_list = []
    if detections.shape != torch.Size([0, 6]):
        for detection in detections:
            confidence = detection[4]
            if float(confidence) < CONFIDENCE_THRESHOLD:
                continue
            xmin, ymin, xmax, ymax = map(int, detection[:4])
            score = float(confidence)
            number_plate_list.append([[xmin, ymin, xmax, ymax, score]])

            # Draw bounding box
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), BOX_COLOR, 2)
            label = "Plate: {:.2f}%".format(score * 100)
            cv2.putText(image, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR, 2)

    return number_plate_list

# ---------- OCR ----------
def recognize_number_plates(image, reader, number_plate_list, current_time=""):
    for i, box in enumerate(number_plate_list):
        xmin, ymin, xmax, ymax, score = box[0]
        np_image = image[ymin:ymax, xmin:xmax]

        # OCR
        detection = reader.readtext(np_image, paragraph=True)
        text = "" if len(detection) == 0 else str(detection[0][1])

        # Print recognized text (console)
        print(f"[{current_time}] Plate {i+1}: {text}")

        # Draw OCR text on frame
        if text != "":
            cv2.putText(image, text, (xmin, ymax + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, TEXT_COLOR, 2)

        # Save cropped plate
        plate_filename = f"plate_{current_time.replace(':','-')}_{i}.jpg"
        plate_path = os.path.join(CROPPED_SAVE_DIR, plate_filename)
        cv2.imwrite(plate_path, np_image)

        # Append result
        result_row = {
            "car_time": current_time,
            "coordinates": [xmin, ymin, xmax, ymax],
            "highest_score": round(score, 4),
            "number_plate_image": plate_path,
            "character_recognized": text
        }
        results_csv.append(result_row)
        results_json.append(result_row)

    return number_plate_list

# ---------- SAVE RESULTS ----------
def save_results():
    # Save CSV
    csv_path = os.path.join(CSV_SAVE_DIR, "number_plate_results.csv")
    with open(csv_path, "w", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["car_time", "coordinates", "highest_score", "number_plate_image", "character_recognized"])
        writer.writeheader()
        writer.writerows(results_csv)

    # Save JSON
    json_path = os.path.join(JSON_SAVE_DIR, "number_plate_results.json")
    with open(json_path, "w") as jsonfile:
        json.dump(results_json, jsonfile, indent=4)

    print(f"Results saved:\nCSV -> {csv_path}\nJSON -> {json_path}\nCropped -> {CROPPED_SAVE_DIR}")

# ---------- MAIN ----------
model = YOLO("/content/drive/MyDrive/MinorProject/Newtrainedmodel/number_plate_model/weights/best.pt")
reader = Reader(['en'], gpu=True)

file_path = "/content/drive/MyDrive/MinorProject/Dataset/car-number-plate/Videos/GTROADCCTV.mp4"
_, file_extension = os.path.splitext(file_path)

# ---- For images ----
if file_extension.lower() in ['.jpg', '.jpeg', '.png']:
    input_img = cv2.imread(file_path)
    current_time = str(datetime.now().strftime("%H:%M:%S"))
    number_plate_list = detect_number_plates(input_img.copy(), model, current_time=current_time)

    if number_plate_list:
        recognize_number_plates(input_img, reader, number_plate_list, current_time=current_time)

    save_results()
    cv2.imwrite(os.path.join(MAIN_SAVE_DIR, "output.jpg"), input_img)

# ---- For videos ----
elif file_extension.lower() in ['.mp4', '.mkv', '.avi', '.wmv', '.mov']:
    print("Processing video...")
    video_cap = cv2.VideoCapture(file_path)
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_video_path = os.path.join(MAIN_SAVE_DIR, "output.mp4")
    writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        success, frame = video_cap.read()
        if not success:
            break

        current_time = str(datetime.now().strftime("%H:%M:%S"))
        number_plate_list = detect_number_plates(frame, model, current_time=current_time)

        if number_plate_list:
            recognize_number_plates(frame, reader, number_plate_list, current_time=current_time)

        writer.write(frame)

    video_cap.release()
    writer.release()
    save_results()
    print(f"Saved video to: {output_video_path}")


Processing video...

0: 384x640 7 number plates, 74.9ms
Speed: 4.4ms preprocess, 74.9ms inference, 89.3ms postprocess per image at shape (1, 3, 384, 640)
[13:36:04] Plate 1: Na54 KKP
[13:36:04] Plate 2: pedePH
[13:36:04] Plate 3: GF67 KVT
[13:36:04] Plate 4: 
[13:36:04] Plate 5: 
[13:36:04] Plate 6: 
[13:36:04] Plate 7: OVIIEAP

0: 384x640 7 number plates, 10.5ms
Speed: 3.4ms preprocess, 10.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[13:36:05] Plate 1: Na54 KKP
[13:36:05] Plate 2: pedePH
[13:36:05] Plate 3: GF67 KVT
[13:36:05] Plate 4: 
[13:36:05] Plate 5: 
[13:36:05] Plate 6: 
[13:36:05] Plate 7: OVIIEAP

0: 384x640 8 number plates, 14.1ms
Speed: 3.3ms preprocess, 14.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
[13:36:06] Plate 1: DEUJERI
[13:36:06] Plate 2: GHIIEAR
[13:36:06] Plate 3: HurstJI
[13:36:06] Plate 4: Nas4 KkP
[13:36:06] Plate 5: GF67 KVT
[13:36:06] Plate 6: 
[13:36:06] Plate 7: 
[13:36:06] Plate 8: Weaaou

0: 384x640 8 num

In [ ]:
from ultralytics import YOLO
from easyocr import Reader
import torch
import cv2
import os
import csv
import json
from datetime import datetime

# ---------- CONFIG ----------
CONFIDENCE_THRESHOLD = 0.5
BOX_COLOR = (0, 255, 0)   # Green box
TEXT_COLOR = (255, 0, 0)  # Blue OCR text

# Create timestamped main folder
current_run_time = datetime.now().strftime("%Y%m%d_%H%M%S")
MAIN_SAVE_DIR = f"/content/drive/MyDrive/MinorProject/Results_csv_Json/{current_run_time}_Detect_recognise"

# Subfolders
CSV_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, "CSV")
JSON_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, "JSON")
CROPPED_SAVE_DIR = os.path.join(MAIN_SAVE_DIR, "Cropped_numberplate")

# Ensure directories exist
os.makedirs(CSV_SAVE_DIR, exist_ok=True)
os.makedirs(JSON_SAVE_DIR, exist_ok=True)
os.makedirs(CROPPED_SAVE_DIR, exist_ok=True)

# Result storage
results_csv = []
results_json = []

# ---------- DETECTION ----------
def detect_number_plates(image, model, current_time=""):
    detections = model.predict(image)[0].boxes.data
    number_plate_list = []
    if detections.shape != torch.Size([0, 6]):
        for detection in detections:
            confidence = detection[4]
            if float(confidence) < CONFIDENCE_THRESHOLD:
                continue
            xmin, ymin, xmax, ymax = map(int, detection[:4])
            score = float(confidence)
            number_plate_list.append([[xmin, ymin, xmax, ymax, score]])

            # Draw bounding box
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), BOX_COLOR, 2)
            label = "Plate: {:.2f}%".format(score * 100)
            cv2.putText(image, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BOX_COLOR, 2)

    return number_plate_list

# ---------- OCR ----------
def recognize_number_plates(image, reader, number_plate_list, current_time=""):
    for i, box in enumerate(number_plate_list):
        xmin, ymin, xmax, ymax, score = box[0]
        np_image = image[ymin:ymax, xmin:xmax]

        # OCR
        detection = reader.readtext(np_image, paragraph=True)
        text = "" if len(detection) == 0 else str(detection[0][1])

        # Print recognized text (console)
        print(f"[{current_time}] Plate {i+1}: {text}")

        # Draw OCR text on frame
        if text != "":
            cv2.putText(image, text, (xmin, ymax + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, TEXT_COLOR, 2)

        # Save cropped plate
        plate_filename = f"plate_{current_time.replace(':','-')}_{i}.jpg"
        plate_path = os.path.join(CROPPED_SAVE_DIR, plate_filename)
        cv2.imwrite(plate_path, np_image)

        # Append result
        result_row = {
            "car_time": current_time,
            "coordinates": [xmin, ymin, xmax, ymax],
            "highest_score": round(score, 4),
            "number_plate_image": plate_path,
            "character_recognized": text
        }
        results_csv.append(result_row)
        results_json.append(result_row)

    return number_plate_list

# ---------- SAVE RESULTS ----------
def save_results():
    # Save CSV
    csv_path = os.path.join(CSV_SAVE_DIR, "number_plate_results.csv")
    with open(csv_path, "w", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["car_time", "coordinates", "highest_score", "number_plate_image", "character_recognized"])
        writer.writeheader()
        writer.writerows(results_csv)

    # Save JSON
    json_path = os.path.join(JSON_SAVE_DIR, "number_plate_results.json")
    with open(json_path, "w") as jsonfile:
        json.dump(results_json, jsonfile, indent=4)

    print(f"Results saved:\nCSV -> {csv_path}\nJSON -> {json_path}\nCropped -> {CROPPED_SAVE_DIR}")

# ---------- MAIN ----------
model = YOLO("/content/drive/MyDrive/MinorProject/Newtrainedmodel/number_plate_model/weights/best.pt")
reader = Reader(['en'], gpu=True)

file_path = "/content/drive/MyDrive/MinorProject/Dataset/car-number-plate/Videos/GTROADCCTV.mp4"
_, file_extension = os.path.splitext(file_path)

# ---- For images ----
if file_extension.lower() in ['.jpg', '.jpeg', '.png']:
    input_img = cv2.imread(file_path)
    current_time = str(datetime.now().strftime("%H:%M:%S"))
    number_plate_list = detect_number_plates(input_img.copy(), model, current_time=current_time)

    if number_plate_list:
        recognize_number_plates(input_img, reader, number_plate_list, current_time=current_time)

    save_results()
    cv2.imwrite(os.path.join(MAIN_SAVE_DIR, "output.jpg"), input_img)

# ---- For videos ----
elif file_extension.lower() in ['.mp4', '.mkv', '.avi', '.wmv', '.mov']:
    print("Processing video...")
    video_cap = cv2.VideoCapture(file_path)
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_video_path = os.path.join(MAIN_SAVE_DIR, "output.mp4")
    writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        success, frame = video_cap.read()
        if not success:
            break

        current_time = str(datetime.now().strftime("%H:%M:%S"))
        number_plate_list = detect_number_plates(frame, model, current_time=current_time)

        if number_plate_list:
            recognize_number_plates(frame, reader, number_plate_list, current_time=current_time)

        writer.write(frame)

    video_cap.release()
    writer.release()
    save_results()
    print(f"Saved video to: {output_video_path}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteProcessing video...

0: 384x640 7 number plates, 42.3ms
Speed: 18.7ms preprocess, 42.3ms inference, 234.3ms postprocess per image at shape (1, 3, 384, 640)
[09:45:08] Plate 1: Na54 KKP
[09:45:08] Plate 2: pedePH
[09:45:08] Plate 3: GF67 KVT
[09:45:08] Plate 4: 
[09:45:08] Plate 5: 
[09:45:08] Plate 6: 
[09:45:08] Plate 7: OVIIEAP

0: 384x640 7 number plates, 10.7ms
Speed: 3.1ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[09:45:10] Plate 1: Na54 KKP
[09:45:10] Plate 2: pedePH
[09:45:10] Plate 3: GF67 KVT
[09:45:10] Plate 4: 
[09:45:10] Plate 5: 
[09:45:10] Plate 6: 
[09:45:10] Plate 7: OVIIEAP

0: 384x640 8 number plates, 10.7ms
Speed: 3.3ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[09:45:10] Plate 1: DEUJERI
[09:45:10] Plate 2: GHIIEAR
[09:45:10] Plate 3: HurstJI
[09:45:10] Plate 4: Nas4 KkP
[09:45:10] Plate 5: GF67 KVT
[09:45

In [3]:
!git --version

git version 2.34.1


In [4]:
!git config --global user.name "mohit9998A"
!git config --global user.email "mohitdutta0407@gmail.com"

In [5]:
%cd /content/drive/MyDrive/MinorProject


/content/drive/MyDrive/MinorProject


In [6]:
!git config --global user.name "mohit9998A"
!git config --global user.email "mohitdutta0407@gmail.com"

# 1. Go to your project folder
%cd /content/drive/MyDrive/MinorProject

# 2. Add all files
!git add .

# 3. Commit files
!git commit -m "Initial commit: ANPR-SYSTEM"

# 4. Rename branch to main
!git branch -M main

# 5. Push to GitHub
!git push -u origin main


/content/drive/MyDrive/MinorProject
^C
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Dataset/
	Newtrainedmodel/
	Results/
	Results_csv_Json/
	TrainedModels/
	detect_and_recognize.ipynb
	preprocessing.ipynb
	training.ipynb

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/mohit9998A/ANPR-SYSTEM.git'
